<a href="https://colab.research.google.com/github/swahareddy/IndianMediaNegativity/blob/master/Indian_News_Negativity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome!

**Vision statement:** <Pending>

### Solution flow:
1. Get news article info from [newsapi.org](https://)
3. Follow the link to actual article and scrape content
4. Use GCP Nat Lang  API to get sentiment from this scraped content
5. Use GCP to asssign category to the article
5. Write to DB
6. Visualise from DB

_______
I have **written an intro** for GCP components here.- https://docs.google.com/document/d/13uFHIBOyLlUpG_pqfvrTX__9fI5yskZvhr6_B_7ViKY/edit?usp=sharing

Acronyms used ahead: GCP- Google cloud platform, BQ - bigQuery
_______

# Task Board

### Fresh work:
* How do I make the job recurring
  * Extend colab timeout
  * Create cron job
  * Python anywhere


### Current DB Structure
p_id,
Publish time,
Write time,
Author,
Source,
Update time,
Title,
Description,
Content,
Sentiment score,
Sentiment magnitude,
Category 1 name,
Category 1 confidence,
Category 2 name,
Category 2 confidence,
Category 3 name,
Category 3 confidence

______
Ignore the follwoing GCP authentication cell for now:



In [ ]:
import time
from datetime import datetime
start_time = time.time()

# Setting up connection with GCP
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "/content/drive/My Drive/Colab Notebooks/maybe-well-do-something-d7263a30102e.json"
project_id="maybe-well-do-something"

# Open BigQuery connection
# !pip install google-cloud-bigquery
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

def run_bq(query):
  query_job = client.query(query)
  results = query_job.result() 
  return results
# stream_bq([(u"India TodayDATE0", 0, u"India Today", u"", u"Punjabgurdwtance.", u"Punjabs: Today", DATETIME(2020, 07, 05, 02, 13, 09), -0.20000000298023224, 3.200000047683716, u"/Sensitive Subjects", 0.6299999952316284, u"/Health", 0.5099999904632568, CURRENT_DATETIME(), CURRENT_DATETIME(), u"https://www.indiatoday.in/india/story/punjab-10-fall-ill-after-consuming-prasad-at-tarn-taran-gurdwara-1697143-2020-07-05")])


In [ ]:
def stream_bq(rows_to_insert): 
  table_id='maybe-well-do-something.news_data_main.news_interim'
  table = client.get_table(table_id)  # Make an API request.
  # rows_to_insert = [(u"India, 0", 1, '1996-10-04 20:23:00', 0.1134, round(3.14144555555,9))]

  errors = client.insert_rows(table, rows_to_insert)  # Make an API request.
  print(errors)
  if errors == []:
      print("New rows have been added.")

# query=
# rows_to_insert=[""]
# stream_bq(rows_to_insert)


## Getting latest articles from [newsapi.org](https://newsapi.org)

Note for alternative APIs:
* Going back in history is something I would definitely want to do and not just start data from when this app is deployed.
* [This](https://ucrawler.app/) is sexy. The user can 
intuitively create their own standard RSS source for whatever website. Allows for **historical data** access!? 
Don't need to limit to news alone, can add blogs etc too.
Has a free plan that allows upto 3 sources per person (but what about RSS fee?ds)
* [This](https://https://webhose.io/news-api/united-kingdom-news-api) offers historical data at a cheap rate
____
The following cell asks for latest 20 articles from each of the 'domains'. It recieves a JSON in response:

In [ ]:

import requests
import pprint
# url = ('http://newsapi.org/v2/top-headlines?'
#        'country=in&'
#        'apiKey=04bb8b8233b348c68003408e02b07443')
# response = requests.get(url)
# pprint.pprint(response.json())
domains=['thehindu.com',
         'hindustantimes.com',
         'ndtv.com',
         'indianexpress.com',
         'theprint.in',
         'thequint.com',
         'scroll.in',
         'scoopwhoop.com',
         'thewire.in',
         'firstpost.com',
         'news18.com' ,
         'indiatoday.in',
         'deccanchronicle.com',
         'republicworld.com',
         'timesnownews.com',
         'aljazeera.com',
         'indiatvnews.com'
        ]

api_results={}
for domain in domains:
  url = ('http://newsapi.org/v2/everything?'
        'domains='+domain+'&'
        'apiKey=04bb8b8233b348c68003408e02b07443')
  response = requests.get(url)
  json_response=response.json()
  pprint.pprint(json_response)
  api_results[domain]=json_response


{'articles': [{'author': None,
               'content': 'The statement deleted from the article on PIB '
                          'contradicted the Indian Council of Medical '
                          "Research's plan to launch a vaccine by 15 August "
                          'this year.\r\n'
                          ' The Press Information Bureau on Sunday published… '
                          '[+4571 chars]',
               'description': 'The statement deleted from the article on PIB '
                              'contradicted the Indian Council of Medical '
                              "Research's plan to launch a vaccine by 15 "
                              'August this year.',
               'publishedAt': '2020-07-05T17:33:25Z',
               'source': {'id': None, 'name': 'Firstpost'},
               'title': 'PIB uploads article claiming none of six Indian '
                        'comp..ve it ready before 2021, later removes mention '
                        'of 

## GCP function definition to calculate sentiment

Takes string as input

Returns dictionary, Eg: `sentiment_out={'Sentiment_mag': 0.20000000298023224, 'Sentiment_score': -0.20000000298023224}`

In [ ]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums

def sample_analyze_sentiment(text_content):
    client = language_v1.LanguageServiceClient()
    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    document = {"content": text_content, "type": type_}
    encoding_type = enums.EncodingType.UTF8 # Available values: NONE, UTF8, UTF16, UTF32
    try:
      response = client.analyze_sentiment(document, encoding_type=encoding_type)
      '''
      # Get overall sentiment of the input document
      print(u"Document sentiment score: {}".format(response.document_sentiment.score))
      print(
          u"Document sentiment magnitude: {}".format(
              response.document_sentiment.magnitude
          )
      )
      '''

      '''
      for sentence in response.sentences:
          print(u"Sentence text: {}".format(sentence.text.content))
          print(u"Sentence sentiment score: {}".format(sentence.sentiment.score))
          print(u"Sentence sentiment magnitude: {}".format(sentence.sentiment.magnitude))
      '''

      # print(u"Language of the text: {}".format(response.language))
      sentiment_output={'Sentiment_score':response.document_sentiment.score,
                        'Sentiment_mag':response.document_sentiment.magnitude,
                        'valid_language':1}
      # print (sentiment_output)
    except:
      sentiment_output={'Sentiment_score':'null',
                        'Sentiment_mag':'null',
                        'valid_language':0}
    return sentiment_output

# GCP function definition to identify relevant categories
Takes string as input (in 20 words)

Returns dictionary like `{'/Sports/Team Sports/Cricket': 0.949999988079071, '/Law & Government/Government': 0.6100000143051147, '/News/Sports News': 0.5199999809265137}`

These are the various possible categories - https://cloud.google.com/natural-language/docs/categories (Overall 700+, but we can bother only with high level categories that are a handful)

In [ ]:
# Get category realted to text
from google.cloud import language

from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

def classify(text, verbose=True):
    language_client = language.LanguageServiceClient()

    document = language.types.Document(
        content=text,
        type=language.enums.Document.Type.PLAIN_TEXT)
    response = language_client.classify_text(document)
    categories = response.categories

    result = {}

    for category in categories:
        result[category.name] = category.confidence

    # if verbose:
        # print(text)
        # for category in categories:
            # print(u'=' * 20)
            # print(u'{:<16}: {}'.format('category', category.name))
            # print(u'{:<16}: {}'.format('confidence', category.confidence))

    return dict(result.items())

# print(classify("Sabiha Bhoomigowda, the outgoing Vice-Chancellor of Karnataka State Akkamahadevi Women’s University, said that she was happy to have completed some of the dream projects of the university during her t"))

# Write to database function

In [ ]:
# Main function to add entries into the databse
# Takes all article information as input, does not return anything

# Converts JSON's timestamp into format compatible with BQ DB 
def break_time(in_time):
  # input 2020-06-24T17:09:04Z
  # output (2008, 12, 25, 05, 30, 00)
  date, time = in_time.split('T')
  year, month, day = date.split('-')
  hour, min, sec = time[:-1].split(':')
  return (f"DATETIME({year}, {month}, {day}, {hour}, {min}, {sec})")

# Remove problematic characters before writing to database
def clean_text(string):
  if string is not None:
    return string.replace("’", "").replace("\n", "").replace("\r", "").replace("\"", "").replace("“", "").replace("”", "").replace("'", "")
  else:
     return ""

def bq_published_time(publishedAt):
  # publishedAt='2020-07-02T08:45:58Z'
  bq_time_format=publishedAt.split('T')[0]+" "+publishedAt.split('T')[1][:-1]
  return(bq_time_format)

def bq_current_time():
  your_unix_timestamp=time.time()
  return(datetime.fromtimestamp(your_unix_timestamp))

columns="pkey, id, Source, Author, Title, Description, Content, Publish_time, Sentiment_score, Sentiment_magnitude, Identified_category_1_name, Identified_category_1_probability, Identified_category_2_name, Identified_category_2_probability, Original_write_time, Update_time, URL"

def bq_streaming_tuple(article, sent_score, sent_mag, article_categories_info, text_content):

  '''
  The 'article' variable passed to this function has structure like this dictionary:
  {'source': {'id': 'the-hindu', 'name': 'The Hindu'}, 
  'author': 'Special Correspondent', 
  'title': 'ICC chairman: Board meets on Thursday to finalise process', 
  'description': 'The International Cricket Council (ICC) Board is set to meet on Thursday to finalise the process of electing Shashank Manohar’s successor as the ICC chairman. The issue has been on the agenda for the', 
  'url': 'https://www.thehindu.com/sport/cricket/icc-chairman-board-meets-on-thursday-to-finalise-process/article31908898.ece', 
  'urlToImage': 'https://www.thehindu.com/static/theme/default/base/img/og-image.jpg', 
  'publishedAt': '2020-06-24T17:09:04Z', 
  'content': 'The International Cricket Council (ICC) Board is set to meet on Thursday to finalise the process of electing Shashank Manohars successor as the ICC chairman. \r\nThe issue has been on the agenda for th… [+742 chars]'}
  '''

  p_id=0
  '''
  # get latest primary key to understand new key
  get_max_id_query="select max(id) from news_data_main.news_master";
  latest_id=(list(run_bq(get_max_id_query))[0][0])
  # print (latest_id)
  if latest_id is None:
    latest_id=0
  # print (latest_id)
  p_id=latest_id+1
  '''

  # print (article)
  source=clean_text(article['source']['name'])
  if source=='Zoom':
    source='Times Now'
  author=clean_text(article['author'])
  if ('staff' in author.lower()) or ('desk' in author.lower()) or (source.lower() in author.lower()) or (author.lower() in source.lower()):
    author=''
  if author==('Press Trust of India'.lower()) or author=='PTI':
    author='Press Trust of India'
  title=clean_text(article['title'])
  description=clean_text(article['description'])

  # content=clean_text(article['content'])
  content=clean_text(text_content)

  publish_ts=bq_published_time((article['publishedAt']))
  url=clean_text(article['url'])

  if len(article_categories_info)>0:
    Identified_category_1_name = str(article_categories_info[0]['name'])
    Identified_category_1_probability = str(article_categories_info[0]['value'])
    if len(article_categories_info)>1:
      Identified_category_2_name = str(article_categories_info[1]['name'])
      Identified_category_2_probability = str(article_categories_info[1]['value'])
    else:
      Identified_category_2_name = ""
      Identified_category_2_probability = "0"        
  else:
    Identified_category_1_name = ""
    Identified_category_1_probability = "0"
    Identified_category_2_name = ""
    Identified_category_2_probability = "0"  
  #   values(225, 'the-hindu', 'Special Correspondent', 'Rane Holdings Q4 net profit plunges 77%', 'Board declares final dividend of ₹4', 'Rane Holdings Ltd, (RHL) the holding company of the Rane Group, reported a 77% drop in its net profit before exceptional items to 73.7 crore for the fourth quarter ended March 2020 on lower volumes a… [+1638 chars]', DATETIME(2020, 06, 24, 17, 15, 25), 0.0, 0.0, , 0, , 0, CURRENT_DATETIME(), CURRENT_DATETIME())
  p_key=f"{source}{str(publish_ts)}{title}"

  values=(p_id, 
          source,
          author, 
          title, 
          description, 
          content, 
          str(publish_ts), 
          round(sent_score, 9), 
          float(sent_mag), 
          Identified_category_1_name, 
          round(float(Identified_category_1_probability),9), 
          Identified_category_2_name, 
          round(float(Identified_category_2_probability),9), 
          str(bq_current_time()), 
          str(bq_current_time()), 
          p_key,
          url)
  # insert_immediate_data_query=f"Insert into news_data_main.news_interim ({columns}) values({values})"

  # Write to interim table
  # print (insert_immediate_data_query)
  # run_bq(insert_immediate_data_query)

  return (values)


# Web Scraper

In [ ]:
import urllib
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import sys

def news_webscraper(url):
  
  # print(url.split("www.")[-1].split("//")[-1].split(".")[0])
  # html = urllib.urlopen(url).read()
  print (url)

  if sys.version_info[0] == 3:
      from urllib.request import urlopen
  else:
      # Not Python 3 - today, it is most likely to be Python 2
      # But note that this might need an update when Python 4
      # might be around one day
      from urllib import urlopen

  url=Request(url, headers={'User-Agent': 'Mozilla/5.0}'})
  # Your code where you can use urlopen
  with urlopen(url) as url:
      s = url.read()

  # print(s)

  soup = BeautifulSoup(s, 'html.parser')

  # kill all script and style elements
  for script in soup(["script", "style"]):
      script.extract()    # rip it out

  # get text
  text = soup.get_text()

  # break into lines and remove leading and trailing space on each
  lines = (line.strip() for line in text.splitlines())
  # break multi-headlines into a line each
  chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
  # drop blank lines
  text = '\n'.join(chunk for chunk in chunks if chunk)

  # print(text)
  full_extract=""
  for line in text.split("\n"):
    # print (line)
    if len(line.split(" "))>20:
      # print (line)
      full_extract=full_extract+"\n"+line

  return (full_extract)


In [ ]:
# !pip3 install newspaper3k
from newspaper import Article
def fetch_article_content(url):
  article = Article(url)
  article.download()
  print(article)
  try:
    article.parse()
    article_content=article.text
  except:
    article_content=""

  return(article_content)


# All processing happens through the following code cell:

In [ ]:
all_queries=[]
sources_information={}
# For each news source:
for source in api_results.keys():
  sources_information[source]={'total_sentiment':0,
                               'total_magnitude':0,
                               'no_of_articles':len(api_results[source]['articles']),
                               'categories':[]}
  
  print ("Number of articles by "+source+" is "+str(len(api_results[source]['articles']))+"\n")
  json_response=api_results[source]
  # For each article for a particular news source
  for article in json_response['articles']:
      categories_info=[]
      # if (article['description'] is not None) and (article['content'] is not None):
      #   print (str(len(article['description']))+" "+str(len(article['content'])))
      # Surprisingly the 'description' of the article is longer than 'content' in JSON
      # text_content=article['description']
      text_content=fetch_article_content(article['url'])
      # print (text_content)

      # Get sentiment score and magnitude . Understand meaning of this at https://cloud.google.com/natural-language/docs/basics#interpreting_sentiment_analysis_values
      # Eg: sentiment_out={'Sentimenjhut_mag': 0.20000000298023224, 'Sentiment_score': -0.20000000298023224}
      sentiment_out=sample_analyze_sentiment(text_content)

      # Handle categories, see all possible categories at https://cloud.google.com/natural-language/docs/categories
      # Eg: {'/Sports/Team Sports/Cricket': 0.949999988079071, '/Law & Government/Government': 0.6100000143051147, '/News/Sports News': 0.5199999809265137}
      if sentiment_out['valid_language']==1:
        article_categories_info=[]
        if len(text_content.split())>20: # Minimum number of words for content classification
          categories_info=classify(text_content) # Returns 2 relevant categories and their confidence scores
          # print (categories_info)
          if len(categories_info)>0:
            categories_info=categories_info.items()
            
            for category_info in categories_info:
              article_category_info={'name':"", 'value':""}
              article_category_info['name']=category_info[0]
              article_category_info['value']=category_info[1]
              article_categories_info.append(article_category_info)
        '''
              # <--COMMENT:: All lines between this and the next comment are for getting  matplotlib visualisation data, can be IGNORED
              category_string=category_info[0]
              split_categories=category_string[1:].split('/')
              if len(split_categories)!=1:
                split_categories=[category_string[1:]]+split_categories
              for split_category in split_categories:
                if split_category not in sources_information[source]['categories']:
                  sources_information[source]['categories']=sources_information[source]['categories']+split_categories
                  print (sources_information[source]['categories'])      

        sources_information[source]['total_sentiment']=sources_information[source]['total_sentiment']+sentiment_out['Sentiment_score']
        sources_information[source]['total_magnitude']=sources_information[source]['total_magnitude']+sentiment_out['Sentiment_mag']
        # ::COMMENT-->
        '''
      # Write all information to database 
      article_query=bq_streaming_tuple(article, sentiment_out['Sentiment_score'], sentiment_out['Sentiment_mag'], article_categories_info, text_content)
      print(article_query)
      # print(all_queries)
      all_queries.append(article_query)
      # run_bq(article_query)
      # all_queries=f"{all_queries}{article_query}"
print(all_queries)
stream_bq(all_queries)
'''
# run_bq(all_queries)
# MERGEEEE into master table (to kill repetitions)
merge_interim_main_query=f"MERGE news_data_main.news_master T USING news_data_main.news_interim S ON T.pkey = S.pkey WHEN NOT MATCHED THEN INSERT ({columns}) VALUES ({columns});"
# run_bq(merge_interim_main_query)

# Clear interim table
clear_interim_query="delete from news_data_main.news_interim where 1=1;"
# run_bq(f"{merge_interim_main_query}{clear_interim_query}")
print ("-------------------------------------------------------------------------")

print("--- %s seconds ---" % (time.time() - start_time))
'''


Number of articles by firstpost.com is 20

https://www.firstpost.com/health/pib-uploads-article-claiming-none-of-six-indian-companies-working-on-covid-19-vaccine-will-have-it-ready-before-2021-later-removes-mention-of-year-8561511.html
(0, 'Firstpost', '', 'PIB uploads article claiming none of six Indian comp..ve it ready before 2021, later removes mention of year - Firstpost', 'The statement deleted from the article on PIB contradicted the Indian Council of Medical Researchs plan to launch a vaccine by 15 August this year.', 'PIB uploads article claiming none of six Indian companies working on COVID-19 vaccine will have it ready before 2021, later removes mention of year - Health News , FirstpostPIB uploads article claiming none of six Indian companies working on COVID-19 vaccine will have it ready before 2021, later removes mention of yearThe statement deleted from the article on PIB contradicted the Indian Council of Medical Researchs plan to launch a vaccine by 15 August this year.

'\n# run_bq(all_queries)\n# MERGEEEE into master table (to kill repetitions)\nmerge_interim_main_query=f"MERGE news_data_main.news_master T USING news_data_main.news_interim S ON T.pkey = S.pkey WHEN NOT MATCHED THEN INSERT ({columns}) VALUES ({columns});"\n# run_bq(merge_interim_main_query)\n\n# Clear interim table\nclear_interim_query="delete from news_data_main.news_interim where 1=1;"\n# run_bq(f"{merge_interim_main_query}{clear_interim_query}")\nprint ("-------------------------------------------------------------------------")\n\nprint("--- %s seconds ---" % (time.time() - start_time))\n'